# Comparison with article

In [1]:
from coupled_cluster.schemes.ccd_sparse import CoupledClusterDoublesSparse
from coupled_cluster.hartree_fock.scf_rhf import scf_rhf
from coupled_cluster.matrix_elements.generate_matrices import (
    get_one_body_elements, get_coulomb_elements,
    get_antisymmetrized_elements, add_spin_to_one_body_elements,
    get_one_body_elements_spin
)
from coupled_cluster.hartree_fock.basis_transformation import (
    transform_one_body_elements, transform_two_body_elements
)

import numpy as np
import time

In [2]:
omega = 0.5
l = 42
n = 2

print ("""
w = {0},
l = {1},
n = {2}
""".format(omega, l, n))

h = omega * get_one_body_elements(l)
t0 = time.time()
u = np.sqrt(omega) * get_coulomb_elements(l)
t1 = time.time()
print ("Time spent creating Coulomb elements: {0} sec".format(t1 - t0))

t0 = time.time()
c, energy = scf_rhf(h.todense(), u, np.eye(l//2), n//2)
t1 = time.time()
print ("Time spent in SCF RHF: {0} sec".format(t1 - t0))

print ("\tRHF Energy: {0}".format(energy))

hi = transform_one_body_elements(h, c)
t0 = time.time()
oi = transform_two_body_elements(u, c)
t1 = time.time()
print ("Time spent transforming two body elements: {0} sec".format(t1 - t0))

_h = add_spin_to_one_body_elements(hi, l)
t0 = time.time()
_u = get_antisymmetrized_elements(l, oi=oi)
t1 = time.time()
print ("Time spent antisymmetrizing two body elements: {0} sec".format(t1 - t0))

t0 = time.time()
ccd_hf = CoupledClusterDoublesSparse(_h, _u, n)
t1 = time.time()
print ("Time spent computing CCD energy with HF basis: {0} sec".format(t1 - t0))
print ("\tCCD (HF) Energy: {0}".format(ccd_hf.compute_energy()))

__h = omega * get_one_body_elements_spin(l)
t0 = time.time()
__u = np.sqrt(omega) * get_antisymmetrized_elements(l)
t1 = time.time()
print ("Time spent getting antisymmetric two body elements: {0} sec".format(t1 - t0))

t0 = time.time()
ccd = CoupledClusterDoublesSparse(__h, __u, n)
t1 = time.time()
print ("Time spent computing CCD energy with HO basis: {0} sec".format(t1 - t0))
print ("\tCCD Energy: {0}".format(ccd.compute_energy()))


w = 0.5,
l = 42,
n = 2

Time spent creating Coulomb elements: 2.5337469577789307 sec
Time spent in SCF RHF: 0.03344917297363281 sec
	RHF Energy: 1.7997479365496687
Time spent transforming two body elements: 0.016511917114257812 sec
Time spent antisymmetrizing two body elements: 0.057221412658691406 sec
Time spent computing CCD energy with HF basis: 2.8599085807800293 sec
	CCD (HF) Energy: 1.6679773214356806
Time spent getting antisymmetric two body elements: 2.5226221084594727 sec
Time spent computing CCD energy with HO basis: 3.1022937297821045 sec
	CCD Energy: 1.7482518328406547


In [3]:
print (h.density)
print (u.density)
print (_u.density)
print (_h.density)
print (__u.density)
print (__h.density)

0.047619047619047616
0.07560121554290651
0.050019024994729565
0.05782312925170068
0.027410132609355157
0.023809523809523808
